In [43]:
%load_ext autoreload
%autoreload 2

from FP567_Lib import *

import tensorflow as tf
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import pathlib
import os
import numpy as np

# improve/change plot appearance
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
'''
After having their html removed, some of the update files are useless. Lets seperate them
'''
move_file_to_if_fnx(
    PATH_TO_RAW_GAME_UPDATES,
    PATH_TO_NOISY_JUNK_UPDATES,
    check_if_init_is_game_updates)

In [ ]:
'''
resoureces/raw_updates/
    by_year_updates
    by_day_updates
    game_updates
are the same updates, just oranized differently. 
Let's use the by_year_updates file and make some specfic yy/mm/dd_update files.
The by_year_updates have a certain form/pattern.
'''
unprocd_year_updates = os.path.join(PATH_TO_RAW_UPDATES, "by_year_updates")
for file in os.listdir(unprocd_year_updates):
    print(file[:4])
    yy = file[2:4]
    lines = open(os.path.join(unprocd_year_updates, file), "r").readlines()
    i = 0
    while i < len(lines):
        toks = lines[i].split()
        if len(toks) == 2:
            maybe_month = toks[1].lower()
            if toks[0] == "##" and maybe_month in MONTHS:
                print(f"\t{maybe_month}")
                path_to_month_folder = os.path.join(
                    PATH_TO_PROCESSED_UPDATES,
                    yy,
                    word_month_to_number_month(maybe_month))
                os.makedirs(path_to_month_folder, exist_ok=True)
                i = i + 2
                update_line_toks = lines[i].split()
                while len(update_line_toks) > 0:
                    d = update_line_toks[1][:-2]
                    print(f"\t\t{d}")
                    # write this update as its own file at yy/m/d.
                    # if in here, maybe_month is now for sure a month
                    with open(os.path.join(path_to_month_folder, f"{d}.txt"), 'w') as f:
                        f.write(" ".join(update_line_toks[3:]))

                    i = i + 1
                    update_line_toks = lines[i].split()
        i = i + 1


In [ ]:
'''
Found another file with some extractable updates that are different than
those that we already have:
resources/Raw_Updates/misc_updates/Combat_Achievements.html.clean
'''
lines = open(os.path.join(PATH_TO_RAW_MISC_UPDATES, "Combat_Achievements.html.clean")).readlines()
i = 0
while i < len(lines):
    toks = lines[i].split()
    if len(toks) == 3 and toks[0] == "Date" and toks[2] == "Changes":
        i = i + 1
        toks = lines[i].split()
        while (toks[0] != "##"):
            # get the date line as a folder path
            d = toks[0]
            path_to_day_folder = os.path.join(
                PATH_TO_PROCESSED_UPDATES_BY_YEAR,
                toks[2][2:4],
                word_month_to_number_month(toks[1]),
                d)
            os.makedirs(path_to_day_folder, exist_ok=True)
            # gather the lines below
            per_day_update_ctr = len(os.listdir(path_to_day_folder))
            i = i + 1
            lines_to_write = []
            line_i = lines[i]
            toks = line_i.split()
            while len(toks) > 0:
                with open(os.path.join(path_to_day_folder, f"{d}_{per_day_update_ctr}.txt"), 'w') as f:
                    f.write(line_i)
                per_day_update_ctr = per_day_update_ctr + 1
                i = i + 1
                line_i = lines[i]
                toks = line_i.split()

            i = i + 1
            toks = lines[i].split()
    i = i + 1